In [4]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
# import pytz module for working with time zone
import pytz, os
import numpy as np

from stockstats import StockDataFrame

# finRL
from finrl.meta.data_processors.processor_alpaca_2 import AlpacaProcessor2
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

# Load env variables
%load_ext dotenv
%dotenv


# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.37


In [2]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# display data on MetaTrader 5 version
print(mt5.version())

(500, 3391, '05 Aug 2022')


In [2]:
# MT5 account infor
MT5_ACCOUNT = os.environ.get('MT5_ACCOUNT')
MT5_PASSWORD = os.environ.get('MT5_PASSWORD')
MT5_SERVER = os.environ.get('MT5_SERVER')

# Alpaca
API_KEY = os.environ.get('API_KEY')
API_SECRET = os.environ.get('API_SECRET')
API_BASE_URL = os.environ.get('API_BASE_URL')
data_url = os.environ.get('data_url')
TIME_INTERVAL_RAW = '4Hour'

# Tick info
TICK_START_DATE='2022-07-01'
TICK_END_DATE='2022-07-31'


TIME_INTERVAL = {
    '1Day': mt5.TIMEFRAME_D1,
    '4Hour': mt5.TIMEFRAME_H4,
    '30Min': mt5.TIMEFRAME_M30,
}

## Alpaca

In [3]:
# alpaca_api = AlpacaProcessor2(
#     start_date=TICK_START_DATE, 
#     end_date=TICK_END_DATE,
#     time_interval=TIME_INTERVAL_RAW,
#     API_KEY=API_KEY, 
#     API_SECRET=API_SECRET, 
#     API_BASE_URL=API_BASE_URL,
# )
# alpaca_api

try:
    alpaca_api = tradeapi.REST(API_KEY, API_SECRET, API_BASE_URL, "v2")
except BaseException:
    raise ValueError("Wrong Account Info!")

In [47]:
timezone = "Europe/Moscow"
start_date = pd.Timestamp(TICK_START_DATE, tz=timezone).isoformat()
print(start_date)
end_date = pd.Timestamp(TICK_END_DATE, tz=timezone).isoformat()
bar_iter = alpaca_api.get_bars("VIXY", TIME_INTERVAL_RAW, start_date, end_date, adjustment='raw').df
# bar_iter = bar_iter.reset_index()
# bar_iter.index = bar_iter.index.datetime.astype('int64')
# bar_iter.index = bar_iter.index.dt.tz_localize('UTC').dt.tz_convert('Europe/Moscow')
# bar_iter['timestamp'] = bar_iter['timestamp'].dt.tz_convert('Asia/Tbilisi')
bar_iter['ts'] = bar_iter.index.values.astype(np.int64) // 10 ** 9
bar_iter

2022-07-01T00:00:00+03:00


,open,high,low,close,volume,trade_count,vwap,ts
timestamp,,,,,,,,
2022-07-01 08:00:00+00:00,18.65,18.71,18.42,18.6200,16253,68,18.522311,1656662400
2022-07-01 12:00:00+00:00,18.59,18.76,18.11,18.3900,2783070,17762,18.418861,1656676800
2022-07-01 16:00:00+00:00,18.39,18.50,17.71,17.7800,4075794,19341,18.029737,1656691200
2022-07-01 20:00:00+00:00,17.78,17.89,17.70,17.7900,50957,91,17.772706,1656705600
2022-07-05 08:00:00+00:00,17.72,18.13,17.70,18.1300,108640,251,17.986797,1657008000
...,...,...,...,...,...,...,...,...
2022-07-28 20:00:00+00:00,14.70,14.76,14.57,14.5800,182005,407,14.654471,1659038400
2022-07-29 08:00:00+00:00,14.60,14.64,14.57,14.6000,39504,153,14.601741,1659081600
2022-07-29 12:00:00+00:00,14.59,14.80,14.47,14.6500,2925678,14422,14.605853,1659096000


In [64]:
def download_data(
        ticker_list, start_date, end_date, time_interval, dataset_path=None
    ) -> pd.DataFrame:

    timezone = "Europe/Moscow"
    start_date = pd.Timestamp(start_date, tz=timezone).isoformat()
    end_date = pd.Timestamp(end_date, tz=timezone).isoformat()
    data_df = pd.DataFrame()
    
    # 2022-07-01T09:30:00+03:00

    for tic in ticker_list:
        barset = alpaca_api.get_bars(
            tic, time_interval, start=start_date, end=end_date
        ).df
        barset["tic"] = tic
        barset = barset.reset_index()
        # data_df = data_df.append(barset)
        data_df = pd.concat([data_df, barset])
    print(("Data before ") + end_date + " is successfully fetched")
    print(data_df.head())

    data_df["timestamp"] = data_df["timestamp"].apply(
        lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
    )

    return data_df

In [48]:
 def add_vix(data):
    vix_df = download_data(["VIXY"], TICK_START_DATE, TICK_END_DATE, TIME_INTERVAL_RAW)
    print(vix_df)
    print(vix_df.head())
    cleaned_vix = self.clean_data(vix_df)
    vix = cleaned_vix[["timestamp", "close"]]
    vix = vix.rename(columns={"close": "VIXY"})


    df = data.copy()
    df = df.merge(vix, on="timestamp")
    df = df.sort_values(["timestamp", "tic"]).reset_index(drop=True)
    return df

## MT5

In [7]:
# now connect to another trading account specifying the password
authorized=mt5.login(login=MT5_ACCOUNT, password=MT5_PASSWORD, server=MT5_SERVER)
if authorized:
    # display trading account data 'as is'
    print(mt5.account_info())
    # display trading account data in the form of a list
    print("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(MT5_ACCOUNT, mt5.last_error()))
 


AccountInfo(login=6106603, trade_mode=0, leverage=100, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=100000.0, credit=0.0, profit=0.0, equity=100000.0, margin=0.0, margin_free=100000.0, margin_level=0.0, margin_so_call=100.0, margin_so_so=50.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Dan Da', server='OANDA-OGM MT5 Demo', currency='USD', company='OANDA Corporation')
Show account_info()._asdict():
  login=6106603
  trade_mode=0
  leverage=100
  limit_orders=0
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=100000.0
  credit=0.0
  profit=0.0
  equity=100000.0
  margin=0.0
  margin_free=100000.0
  margin_level=0.0
  margin_so_call=100.0
  margin_so_so=50.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=Dan

In [68]:
eurusd_rates = mt5.copy_rates_range("EURUSD", TIME_INTERVAL[TIME_INTERVAL_RAW], datetime(2022,7,1,0), datetime(2022,7,31,0))

In [69]:
ticks_frame = pd.DataFrame(eurusd_rates)

# Convert timestamp to datetime
ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s')

# Add tick name
ticks_frame['tic'] = 'EURUSD'

# Rename tick_volume to volume
ticks_frame = ticks_frame.rename(columns={"time": "date", "tick_volume": "volume"}, errors="raise")

# Drop column "real_volume"
ticks_frame = ticks_frame.drop(columns=['real_volume'])

# Wrap the stockstatic dataframe
stock_df = StockDataFrame.retype(ticks_frame)
stock_df

,open,high,low,close,volume,spread,tic
date,,,,,,,
2022-07-01 08:00:00,1.04534,1.04701,1.04322,1.04584,35183,10,EURUSD
2022-07-01 12:00:00,1.04583,1.04737,1.04061,1.04062,32689,10,EURUSD
2022-07-01 16:00:00,1.04062,1.04195,1.03649,1.04007,52002,10,EURUSD
2022-07-01 20:00:00,1.04007,1.04325,1.03928,1.04259,13812,10,EURUSD
2022-07-04 00:00:00,1.04195,1.04438,1.04163,1.04335,10229,10,EURUSD
...,...,...,...,...,...,...,...
2022-07-29 04:00:00,1.01974,1.02244,1.01906,1.02086,16480,10,EURUSD
2022-07-29 08:00:00,1.02086,1.02539,1.02022,1.02210,42906,10,EURUSD
2022-07-29 12:00:00,1.02210,1.02453,1.01738,1.01767,43316,10,EURUSD


In [65]:
add_vix(stock_df)

Data before 2022-07-31T00:00:00+03:00 is successfully fetched
                  timestamp   open   high      low  close   volume  \
0 2022-07-01 04:00:00+00:00  18.43  18.76  17.7100  17.78  6928989   
1 2022-07-05 04:00:00+00:00  18.45  18.80  17.7150  17.77  8398902   
2 2022-07-06 04:00:00+00:00  17.81  18.08  17.3900  17.48  5287606   
3 2022-07-07 04:00:00+00:00  17.37  17.40  17.0301  17.29  4511801   
4 2022-07-08 04:00:00+00:00  17.35  17.35  16.7950  16.93  5432839   

   trade_count       vwap   tic  
0        37409  18.185394  VIXY  
1        45261  18.275588  VIXY  
2        29148  17.767430  VIXY  
3        22619  17.216084  VIXY  
4        28014  17.025150  VIXY  
              timestamp   open     high      low  close   volume  trade_count  \
0   2022-07-01 04:00:00  18.43  18.7600  17.7100  17.78  6928989        37409   
1   2022-07-05 04:00:00  18.45  18.8000  17.7150  17.77  8398902        45261   
2   2022-07-06 04:00:00  17.81  18.0800  17.3900  17.48  5287606      

NameError: name 'self' is not defined

In [ ]:
stock_df[["change", "rate", "close_-1_d", "log-ret"]]

In [ ]:
stock_df[["close", "close_14_sma", "close_50_sma", "close_200_sma" ]]

In [ ]:
stock_df[["close", "close_14_sma", "close_50_sma", "close_200_sma" ]].plot(title="Close SMA");

In [ ]:
stock_df[["tick_volume", "tick_volume_14_sma", "tick_volume_xu_tick_volume_14_sma" ]]

In [ ]:
stock_df[["tick_volume", "tick_volume_14_sma", ]].plot(title="Bollinger Bands example");